In [ ]:
import numpy as np
import os.path as osp
import cv2
import time
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.backend as K
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

import modules.utils
import modules.detector
from modules.geometry import RectsImage
from modules.images_viewer import ImagesViewer
from modules.dataset import Dataset
from modules.detector import FCNDetector
from modules.images_viewer import ImagesViewer
from modules.quality import compute_quality, compute_average_precision, find_optimal_threshold, get_precision_recall_curve
import modules.models.loader as loader
import config

In [ ]:
def show(image, size_w=8, size_h=8):
    image = image[:, :, ::-1]
    plt.figure(figsize=(size_w, size_h))
    plt.imshow(image)

# Input

In [ ]:
detector = FCNDetector()

In [ ]:
dataset = Dataset()

# Show image with rects

In [ ]:
image_data = dataset.images_data[0]
if not config.load_all_images_to_ram:
    image_data.load()
image = image_data.image.copy()
for true_rect in image_data.rects:
    true_rect.draw(image, (0, 255, 0), 5)

mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
nms_heat_map = detector.heat_map_nms(mask)
rects = detector.obtain_rects(nms_heat_map, mask)
reduced_rects = FCNDetector.rects_nms(rects)
for rect in reduced_rects:
    rect.draw(image, (255, 0, 0), 3)
print(len(reduced_rects))
show(image, 22, 22)
if not config.load_all_images_to_ram:
    image_data.release()

# Check NMS

In [ ]:
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    if len(image_data.rects) == len(reduced_rects):
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {} != {}".format(image_data.image_name, len(image_data.rects), len(reduced_rects)))
        
    if not config.load_all_images_to_ram:
        image_data.release()

# Check Quality

In [ ]:
quality_objects_union = []
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    quality_objects = compute_quality(image_data.rects, reduced_rects)
    quality_objects_union.extend(quality_objects)
    ap_rate = compute_average_precision(quality_objects)
    th, best_precision, best_recall, best_f1 = find_optimal_threshold(quality_objects)
    
    if ap_rate > 0.9999 and best_precision > 0.9999 and best_recall > 0.9999:
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {}".format(image_data.image_name, best_recall))
        
    if not config.load_all_images_to_ram:
        image_data.release()

In [ ]:
true_labels = []
predictions = []
for q in quality_objects:
    true_labels.append(q.label)
    predictions.append(q.prediction)
precision, recall, thresholds = precision_recall_curve(true_labels, predictions)

In [ ]:
recall

In [ ]:
find_optimal_threshold(quality_objects_union)

# Estimate Quality

In [ ]:
K.clear_session()
fcn_model_module = loader.get_fcn_model_module()
fcn_model = fcn_model_module.FCNModel()
detector = FCNDetector(fcn_model.model, osp.join(fcn_model.weights_dir, 'best_weights.hdf5'))

#### Check patching

In [ ]:
for i, image_data in enumerate(dataset.images_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    image = image_data.image
    heat_map = detector.predict_heatmap_by_patching(image)
    heat_map_full = detector.predict_heatmap(image)
    heat_map = heat_map[:heat_map_full.shape[0], :heat_map_full.shape[1], :]
    dif = np.abs(heat_map_full - heat_map)
    dif_mask = dif > 0.02
    nonzero_pixels_count = len(np.nonzero(dif_mask)[0])
    print("{} {}: \t{}".format(i, image_data.image_name, nonzero_pixels_count))
    if not config.load_all_images_to_ram:
        image_data.release()

### One Image Processing

In [ ]:
image_data = dataset.images_data[25]
if not config.load_all_images_to_ram:
    image_data.load()
print(image_data.image_name)
image = image_data.image

In [ ]:
%%time
heat_map = detector.predict_heatmap_by_patching(image)

In [ ]:
nms_heat_map = detector.heat_map_nms(heat_map)

In [ ]:
rects = detector.obtain_rects(nms_heat_map, heat_map)
reduced_rects = FCNDetector.rects_nms(rects)

In [ ]:
quality_objects = compute_quality(dataset.images_data[0].rects, reduced_rects)

In [ ]:
ap_rate = compute_average_precision(quality_objects)
ap_rate

In [ ]:
threshold, precision, recall, f1_score = find_optimal_threshold(quality_objects)

In [ ]:
threshold, precision, recall, f1_score

In [ ]:
precision, recall, thresholds = get_precision_recall_curve(quality_objects)

In [ ]:
for i in range(len(precision)):
    print("{} {}".format(precision[i], recall[i]))

In [ ]:
filtered_rects = [rect for rect in reduced_rects if rect.score > threshold]
print(len(reduced_rects))
print(len(filtered_rects))

In [ ]:
image_with_rects = image_data.image.copy()
for true_rect in image_data.rects:
    true_rect.draw(image_with_rects, (0, 255, 0), 5)
for rect in filtered_rects:
    rect.draw(image_with_rects, (255, 0, 0), 3)

In [ ]:
images_viewer = ImagesViewer()
images_viewer.set_images([image, image_with_rects, heat_map*255, nms_heat_map*255])
images_viewer.run()

#### Compare overalapping prediction and whole image prediction

In [ ]:
%%time
heat_map1 = detector.predict_heatmap(image)

In [ ]:
show(heat_map1)

In [ ]:
%%time
heat_map = detector.predict_heatmap_by_patching(image)

In [ ]:
show(heat_map)

In [ ]:
dif = np.abs(heat_map - heat_map1)
dif.max(), dif.mean()

In [ ]:
images_viewer = ImagesViewer()
images_viewer.set_images([image_data.image, heat_map*255, heat_map1*255, dif*255])
images_viewer.run()

### Quality over the all images

In [ ]:
def get_quality_objects(image_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = detector.predict_heatmap(image_data.image)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    quality_objects = compute_quality(image_data.rects, reduced_rects)
    ap_rate = compute_average_precision(quality_objects)
    
    print("{}:\t {}".format(image_data.image_name, ap_rate))

    if not config.load_all_images_to_ram:
        image_data.release()
        
    return quality_objects

In [ ]:
train_quality_objects = []
for i in tqdm.tqdm_notebook(dataset.train_indices):
    image_data = dataset.images_data[i]
    train_quality_objects.extend(get_quality_objects(image_data))

test_quality_objects = []
for i in tqdm.tqdm_notebook(dataset.test_indices):
    image_data = dataset.images_data[i]
    test_quality_objects.extend(get_quality_objects(image_data))    

train_ap_rate = compute_average_precision(train_quality_objects)
print("Train AP: {}".format(train_ap_rate))
test_ap_rate = compute_average_precision(test_quality_objects)
print("Test AP: {}".format(test_ap_rate))

In [ ]:
threshold, best_precision, best_recall, best_f1 = find_optimal_threshold(train_quality_objects)
print("Train F1 score {}, precision {}, recall {}".format(best_f1, best_precision, best_recall))
threshold, best_precision, best_recall, best_f1 = find_optimal_threshold(test_quality_objects)
print("Test F1 score {}, precision {}, recall {}".format(best_f1, best_precision, best_recall))